# Federated Meta-Learning Lab using MAML
This notebook demonstrates how to set up a federated meta-learning framework using Model-Agnostic Meta-Learning (MAML) with images from three subfolders in the main dataset directory.

In [1]:
# Import required libraries
import os
import random
from glob import glob
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# Define the dataset path and parameters
DATASET_PATH = "D:\\Artificial Intelligence\\Machine Learning\\PhD - Resource Aware Machine Learning\\FederatedMetaLearning\\Dataset_Fruit"  # Main dataset folder with subfolders for classes
IMAGE_SIZE = (512, 512)   # Resize all images to 224x224
NUM_CLASSES = 3           # Three classes
CLIENTS = 3               # Number of federated clients
EPOCHS = 100
BATCH_SIZE = 10
INNER_LR = 0.05           # Learning rate for inner loop
META_LR = 0.005           # Learning rate for meta-learning

In [3]:
def load_images_from_folder(folder_path, class_label, image_size):
    """Load images from a specific folder and assign a class label."""
    images = []
    labels = []
    image_files = glob(os.path.join(folder_path, "*"))
    for img_file in image_files:
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):  # Filter for image files
            try:
                img = Image.open(img_file).convert("RGB").resize(image_size)
                images.append(np.array(img) / 255.0)  # Normalize pixel values
                labels.append(class_label)
            except Exception as e:
                print(f"Error loading image {img_file}: {e}")
    return images, labels

In [4]:
# Iterate through subfolders to load data for three classes
def load_dataset(dataset_path, image_size, num_classes):
    all_images = []
    all_labels = []
    subfolders = sorted(os.listdir(dataset_path))[:num_classes]  # First three classes
    for class_idx, subfolder in enumerate(subfolders):
        folder_path = os.path.join(dataset_path, subfolder)
        images, labels = load_images_from_folder(folder_path, class_idx, image_size)
        all_images.extend(images)
        all_labels.extend(labels)
    return np.array(all_images), np.array(all_labels)

In [5]:
# Load the dataset
print("Loading dataset...")
images, labels = load_dataset(DATASET_PATH, IMAGE_SIZE, NUM_CLASSES)
labels = to_categorical(labels, num_classes=NUM_CLASSES)

Loading dataset...


In [6]:
# Split the data into federated clients
def split_federated_clients(images, labels, num_clients):
    """Distribute the dataset among federated clients."""
    clients_data = []
    data_size = len(images)
    indices = np.arange(data_size)
    random.shuffle(indices)
    split_size = data_size // num_clients
    for i in range(num_clients):
        start_idx = i * split_size
        end_idx = start_idx + split_size if i != num_clients - 1 else data_size
        client_images = images[indices[start_idx:end_idx]]
        client_labels = labels[indices[start_idx:end_idx]]
        clients_data.append((client_images, client_labels))
    return clients_data

clients_data = split_federated_clients(images, labels, CLIENTS)
print(f"Data successfully split among {CLIENTS} clients.")

Data successfully split among 3 clients.


In [7]:
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [8]:
def maml_step(model, data, labels, inner_lr):
    with tf.GradientTape() as tape:
        predictions = model(data)
        loss = tf.keras.losses.categorical_crossentropy(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    k = 0
    for var in model.trainable_variables:
        var.assign_sub(inner_lr * gradients[k])
        k += 1

In [9]:
# Meta-Learning simulation: Iterate through clients and update a global model
global_model = create_model((*IMAGE_SIZE, 3), NUM_CLASSES)
global_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=META_LR), loss='categorical_crossentropy', metrics=['accuracy'])

for round_num in range(1, EPOCHS + 1):
    print("*" * 30)
    print(f"Federated Round {round_num}")
    
    for client_idx, client_data in enumerate(clients_data):
        print(f"Training on Client {client_idx + 1}")
        # Clone the global model for each client
        local_model = tf.keras.models.clone_model(global_model)
        local_model.set_weights(global_model.get_weights())
        
        # Split client data into support and query sets
        support_data, query_data, support_labels, query_labels = train_test_split(
            client_data[0], client_data[1], test_size=0.3)
        
        # Inner loop: Update local model on support set
        maml_step(local_model, support_data, support_labels, INNER_LR)
        
        # Evaluate local model accuracy on query set
        local_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=META_LR), loss='categorical_crossentropy', metrics=['accuracy'])
        local_eval_loss, local_eval_accuracy = local_model.evaluate(query_data, query_labels, verbose=0)
        print(f"Client {client_idx + 1} after MAML step: {local_eval_accuracy * 100:.2f}%")
        
        global_eval_loss, global_eval_accuracy = global_model.evaluate(query_data, query_labels, verbose=0)
        print(f"Global Accuracy: {global_eval_accuracy * 100:.2f}%")
        
        if local_eval_accuracy > global_eval_accuracy :
            # Evaluate on query set and compute meta-gradient
            print("==== Update Global!")
            with tf.GradientTape() as tape:
                query_predictions = local_model(query_data)
                query_loss = tf.keras.losses.categorical_crossentropy(query_labels, query_predictions)
            query_gradients = tape.gradient(query_loss, global_model.trainable_variables)

            # Apply meta-gradient update
            for var, grad in zip(global_model.trainable_variables, query_gradients):
                if grad is not None:
                    var.assign_sub(META_LR * grad)


c:\Users\Cloud Tech\anaconda3\envs\py310\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


******************************
Federated Round 1
Training on Client 1
Client 1 after MAML step: 50.00%
Global Accuracy: 50.00%
Training on Client 2
Client 2 after MAML step: 62.50%
Global Accuracy: 25.00%
==== Update Global!
Training on Client 3
Client 3 after MAML step: 37.50%
Global Accuracy: 25.00%
==== Update Global!
******************************
Federated Round 2
Training on Client 1
Client 1 after MAML step: 37.50%
Global Accuracy: 37.50%
Training on Client 2
Client 2 after MAML step: 75.00%
Global Accuracy: 12.50%
==== Update Global!
Training on Client 3
Client 3 after MAML step: 37.50%
Global Accuracy: 50.00%
******************************
Federated Round 3
Training on Client 1
Client 1 after MAML step: 37.50%
Global Accuracy: 62.50%
Training on Client 2
Client 2 after MAML step: 25.00%
Global Accuracy: 50.00%
Training on Client 3
Client 3 after MAML step: 12.50%
Global Accuracy: 0.00%
==== Update Global!
******************************
Federated Round 4
Training on Client 1
Cl

In [13]:
# Evaluate the global model
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.5)
test_loss, test_accuracy = global_model.evaluate(test_images, test_labels, verbose=1)
print(f"Global Model Accuracy: {test_accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5071 - loss: 1.0738 
Global Model Accuracy: 52.63%
